In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/poorn/Downloads/crime-in-los-angeles/Crime.csv')

In [2]:
dataset=pd.DataFrame(data);

In [3]:
dataset['year'] = pd.DatetimeIndex(dataset['Date Occurred']).year 

In [4]:
sankey = pd.DataFrame({'count' : dataset.groupby(['year','Victim Sex','Area Name'])['Area ID'].count()}).reset_index()

In [21]:
import pandas as pd
import plotly
import chart_studio.plotly as py
def generateSankey(datas,column=[],values='',title='Crime Analysis'):
    
    colors = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelLists = []
    colorNumLists = []
    for col in column:
        labelListTemps =  list(set(datas[col].values))
        colorNumLists.append(len(labelListTemps))
        labelLists = labelLists + labelListTemps
        
    
    labelLists = list(dict.fromkeys(labelLists))
    
    
    colorLists = []
    for id, colorNums in enumerate(colorNumLists):
        colorLists = colorLists + [colors[id]]*colorNums
        
    
    for i in range(len(column)-1):
        if i==0:
            sourceTargetDfs = datas[[column[i],column[i+1],values]]
            sourceTargetDfs.columns = ['source','target','count']
        else:
            tempDfs = datas[[column[i],column[i+1],values]]
            tempDfs.columns = ['source','target','count']
            sourceTargetDfs = pd.concat([sourceTargetDfs,tempDfs])
        sourceTargetDfs = sourceTargetDfs.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    
    sourceTargetDfs['sourceID'] = sourceTargetDfs['source'].apply(lambda x: labelLists.index(x))
    sourceTargetDfs['targetID'] = sourceTargetDfs['target'].apply(lambda x: labelLists.index(x))
    
    
    fin = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelLists,
          color = colorLists
        ),
        link = dict(
          source = sourceTargetDfs['sourceID'],
          target = sourceTargetDfs['targetID'],
          value = sourceTargetDfs['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 12
        )
    )
       
    figs = dict(data=[fin], layout=layout)
    return figs



fig = generateSankey(sankey,column=['year','Victim Sex','Area Name'],values= 'count',title='Crime Analysis')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'